In [1]:
import pandas as pd
import joblib

log_model = joblib.load("../models/logistic_regression.pkl")
nb_model = joblib.load("../models/naive_bayes.pkl")
vectorizer = joblib.load("../models/tfidf_vectorizer.pkl")

print("Models and vectorizer loaded sucessfully.")

Models and vectorizer loaded sucessfully.


In [2]:
from sklearn.model_selection import train_test_split

df = pd.read_csv("../data/processed/reddit_clean.csv")

X = df["clean_text"]
y = df["label"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

print("Test case size:", X_test.shape)

Test case size: (465418,)


In [3]:
X_test_tfidf = vectorizer.transform(X_test)

log_preds = log_model.predict(X_test_tfidf)
nb_preds = nb_model.predict(X_test_tfidf)

print("Predictions generated.")

Predictions generated.


In [4]:
error_df = pd.DataFrame({
    "text": X_test.values,
    "true_label": y_test.values,
    "log_pred": log_preds,
    "nb_pred": nb_preds
})

error_df.head()

,text,true_label,log_pred,nb_pred
0,grandmother help mom every often spend time tr...,0.0,1,0
1,many people believe dumb star judge personalit...,0.0,0,0
2,bored need social interaction,0.0,0,0
3,reddit premium gay,0.0,0,0
4,little trolling,0.0,0,0


In [5]:
# Analyzing False Negative
log_fn = error_df[
    (error_df["true_label"] == 1) &
    (error_df["log_pred"] == 0)
]

print("Logistic Regression False Negatives:", len(log_fn))
log_fn.sample(5)

Logistic Regression False Negatives: 9636


,text,true_label,log_pred,nb_pred
313352,personally guy hero wanted let know hope conti...,1.0,0,0
320332,screamed take anymore house kind relieving ngl,1.0,0,0
348711,shit happens every fucking time like know ment...,1.0,0,0
313154,monday start red ribbon week anti drug thing h...,1.0,0,0
328167,really hard time today please send thing make ...,1.0,0,0


In [6]:
nb_fn = error_df[
    (error_df["true_label"] == 1) &
    (error_df["nb_pred"] == 0)
]

print("Naive Bayes False Negatives:", len(nb_fn))
nb_fn.sample(5)

Naive Bayes False Negatives: 28550


,text,true_label,log_pred,nb_pred
306023,think okay hit like train cycle,1.0,1,0
65316,everyone suicidal thought watching video inter...,1.0,1,0
259320,watch show play game ignore world around hard ...,1.0,1,0
220189,really care,1.0,0,0
271843,life point impossible reach happiness least be...,1.0,1,0


### False Negative Analysis

False negatives represent depressed posts incorrectly classified as normal.

- Logistic Regression shows **fewer false negatives**, meaning it detects
  depression more reliably.
- Naive Bayes misses more depressed posts, making it riskier for
  mental-health-related applications.

In depression detection, minimizing false negatives is more important
than minimizing false positives.

### Final Model Choice

Although both models achieve similar accuracy (~92%):

- **Logistic Regression** achieves much higher recall for depressed posts.
- **Naive Bayes** is more conservative but misses many depressed users.

For depression detection, **Logistic Regression is the preferred model**
because it minimizes false negatives, which is critical in healthcare
and mental well-being applications.

### *Model Explainability*

In [8]:
# This gives us the actual words used by TF-IDF.
feature_names = vectorizer.get_feature_names_out()

print("Number of features:", len(feature_names))
feature_names[:20]

Number of features: 10000


array(['aa', 'aaa', 'aaaa', 'aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa', 'ab',
       'abandon', 'abandoned', 'abandoning', 'abandonment', 'abilify',
       'ability', 'able', 'abnormal', 'abomination', 'abort', 'aborted',
       'abortion', 'abroad', 'abruptly', 'absence'], dtype=object)

In [11]:
coefficients = log_model.coef_[0]

print("Coefficient shape:", coefficients.shape)

# One coefficient per word
# Positive → depressed
# Negative → normal

Coefficient shape: (10000,)


In [12]:
coef_df = pd.DataFrame({
    "word": feature_names,
    "weight": coefficients
})

coef_df.head()

,word,weight
0,aa,-0.118351
1,aaa,-0.756722
2,aaaa,-0.858383
3,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa,-0.121965
4,ab,-1.292852


In [13]:
top_depressed_words = coef_df.sort_values(
    by="weight", ascending=False
).head(20)

top_depressed_words

,word,weight
2302,depression,12.226123
8608,suicide,8.928568
8691,sw,7.709509
5482,mg,7.191065
2299,depressed,6.973586
378,antidepressant,6.921134
8607,suicidal,6.811209
6840,psychiatrist,6.643531
2303,depressive,6.639298
8609,suicidewatch,6.044951


In [14]:
top_normal_words = coef_df.sort_values(
    by="weight", ascending=True
).head(20)

top_normal_words

,word,weight
8818,teenager,-13.206263
2037,crush,-9.160164
5524,minecraft,-8.941559
5432,meme,-8.747694
2552,dm,-8.114542
4831,karma,-7.691362
8811,ted,-7.650465
2468,discord,-7.406808
3384,flair,-6.998209
4226,horny,-6.617559


### Model Explainability (Logistic Regression)

The Logistic Regression model assigns a weight to each word based on its
contribution to predicting depression.

- Words with **positive weights** increase the likelihood of a post being
  classified as depressed.
- Words with **negative weights** reduce the likelihood of depression.

#### Depression-Indicative Words
The model strongly associates words such as *worthless*, *hopeless*,
*alone*, and *tired* with depressed posts.

#### Normal / Non-Depressed Words
Words like *happy*, *excited*, *love*, and *great* are associated with
normal mental states and reduce the probability of depression.

This makes the model interpretable and suitable for sensitive applications
like mental health analysis.